
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src=https://raw.githubusercontent.com/aestaire/ml_workshop/refs/heads/main/files/images/hands-on.png>
</div>

#Despliegue del Modelo en Kubernetes
Hemos entrenado un modelo en Databricks, lo hemos registrado en MLflow Model Registry, y ahora queremos sacarlo del entorno Databricks para.

En este notebook vamos a recorrer todo ese camino paso a paso, desde la autenticación hasta el despliegue.

Aprenderás a:
* Conectarte a tu workspace de Databricks desde tu entorno on-prem
* Descargar un modelo registrado en MLflow
* Ejecutarlo localmente y hacer inferencias
* Empaquetarlo en Docker
* Desplegarlo en Kubernetes con un manifiesto simple

## 1. Prerrequisitos

Antes de empezar asegúrate de tener instalado lo siguiente:

* Python 3.8+ y las librerías necesarias: (pip install --upgrade mlflow databricks-cli)

También necesitas:
* Docker Desktop (para construir y correr contenedores).
* Acceso a tu workspace de Databricks.
* Un Personal Access Token (PAT) o autenticación SSO configurada.

📚 Referencias oficiales:
* [Databricks CLI (Unified Auth)](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/cli/)
* [MLflow Models overview](https://mlflow.org/docs/latest/ml/model/)

## 2. Autenticación con Databricks CLI

Vale, lo primero es poder hablar con nuestro workspace desde fuera.
El Databricks CLI es la forma más sencilla de autenticarnos desde terminal o scripts locales.

🔸 Paso 1 — Login
`databricks auth login \
  --host https://<tu-workspace>.azuredatabricks.net`


Esto te pedirá abrir un enlace o pegar tu token.

🔸 Paso 2 — Comprobar conexión
`databricks workspace ls /`


Si ves la lista de carpetas de tu workspace, ¡ya estás dentro! 🎉

📘 Docs: [CLI Authentication](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/cli/authentication)

## 3. Configurar entorno Python on-prem

Ahora que tenemos acceso al workspace, vamos a preparar MLflow para apuntar al tracking server de Databricks.

En una celda:
```
import mlflow
# Apunta al tracking server de Databricks
mlflow.set_tracking_uri("databricks")

# (Opcional) Si usas token, puedes definirlo como variable de entorno
# %env DATABRICKS_HOST=https://<tu-workspace>.azuredatabricks.net
# %env DATABRICKS_TOKEN=<tu_token>
```

📘 Docs: [MLflow Tracking URIs](https://mlflow.org/docs/latest/ml/tracking/#where-runs-are-recorded)

## 4. Entendiendo cómo se referencia un modelo

En MLflow podemos apuntar a un modelo de varias formas:	

| Tipo | Ejemplo | Uso |
| -- | -- | -- |
| Por versión      | models:/catalogo.esquema.modelo/version  | Para reproducibilidad exacta |
| Por alias | models:/catalogo.esquema.modelo@alias              | Ideal para despliegues |

📘 Docs: [MLflow Model Registry URIs](https://mlflow.org/docs/latest/ml/model-registry/)

## 5. Descargar el modelo desde MLflow

Vale, ahora sí, vamos a descargar el modelo localmente.
```
import mlflow

mlflow.set_registry_uri("databricks-uc")  

# Descarga los artefactos
model_uri = "models:/<CATALOGO>.<ESQUEMA>.<MODELO>/1"  # o "@Champion"
local_dir = "/tmp/mi_modelo"

mlflow.artifacts.download_artifacts(artifact_uri=model_uri, dst_path=local_dir))
```
Esto te deja una carpeta con todo: el modelo, el MLmodel, dependencias y metadata.

📘 Docs: [download_artifacts](https://mlflow.org/docs/latest/api_reference/python_api/mlflow.artifacts.html)

## 6. Cargar el modelo y hacer inferencia local

Una vez descargado, podemos cargarlo directamente en memoria y probarlo.

```
import mlflow
import pandas as pd

# Carga desde ruta local y predice
import pandas as pd
model = mlflow.pyfunc.load_model(local_dir)
X = pd.DataFrame([{"feature1": 0.3, "feature2": 1.2}])
print(model.predict(X))

# Ejemplo de predicción
X = pd.DataFrame([
    {"feature1": 0.3, "feature2": 1.2},
    {"feature1": 0.8, "feature2": 3.1},
])

preds = model.predict(X)
```

📘 Docs: [mlflow.pyfunc.load_model](https://mlflow.org/docs/latest/api_reference/python_api/mlflow.pyfunc.html)

## 7. Empaquetar el modelo como contenedor Docker

Como ya tienes el modelo descargado a disco, puedes construir el contenedor directamente desde esa carpeta, para convertirlo en una API lista para servir inferencias.

```
mlflow models build-docker \
  -m "local_dir" \ #apunta al path local donde está el modelo
  -n "<tu_repo>/<tu_imagen>:v1" \ #nombre y etiqueta de la imagen Docker que se generará
  --enable-mlserver
```
Esto construye una imagen con:

* Python runtime

* El modelo

* MLServer (servidor de inferencia REST)

Puedes verla en tu lista de imágenes locales:

* docker images | grep <tu_imagen>


📘 Docs: [Deploy MLflow Models to Docker](https://mlflow.org/docs/latest/ml/model/#build-a-docker-image)



## 8. Probar el contenedor localmente
1) Ejecuta el contenedor

`docker run --rm -p 8080:8080 "<tu_repo>/<tu_imagen>:v1"`

2) En otra terminal, envía una predicción:

```
curl -X POST http://localhost:8080/invocations \
  -H "Content-Type: application/json" \
  -d '{"inputs": [{"feature1": 0.3, "feature2": 1.2}]}'
```
Si obtienes un JSON con predicciones, ya tienes tu modelo sirviendo en tu máquina local 🚀

Docs: [Serving models locally](https://mlflow.org/docs/latest/ml/model/#local-deployment)



## 9. Desplegar el modelo en Kubernetes

Ahora vamos a llevar ese mismo contenedor a Kubernetes con un manifiesto básico.

Crea un archivo k8s-mlflow-model.yaml con este contenido:
```
apiVersion: apps/v1
kind: Deployment
metadata:
  name: mlflow-model
spec:
  replicas: 2
  selector:
    matchLabels:
      app: mlflow-model
  template:
    metadata:
      labels:
        app: mlflow-model
    spec:
      containers:
      - name: mlflow-model
        image: <tu_repo>/<tu_imagen>:v1
        ports:
        - containerPort: 8080
---
apiVersion: v1
kind: Service
metadata:
  name: mlflow-model-svc
spec:
  selector:
    app: mlflow-model
  ports:
  - port: 80
    targetPort: 8080
```

Despliega con:
```
kubectl apply -f k8s-mlflow-model.yaml
kubectl get pods
```

Y para probarlo:
```
kubectl port-forward svc/mlflow-model-svc 8080:80
curl -X POST http://localhost:8080/invocations \
  -H "Content-Type: application/json" \
  -d '{"inputs": [{"feature1": 0.3, "feature2": 1.2}]}'
```

📘 Docs: [Deploy MLflow Model to Kubernetes](https://mlflow.org/docs/latest/ml/model/#deployment)

### ¡Felicidades! Ya tendríamos funcionando nuestro modelo en local

Ahora vamos a continuar aprendiendo sobre las [AI functions]($./03_AI_functions)
